# Merging Blocks using Plot3D
With large meshes, it can be more efficient to minimize the number of blocks and send larger blocks to the CPU or GPU instead. This tutorial shows how to perform that combination automatically or manually

## Download an example mesh 

In [ ]:
!wget https://nasa-public-data.s3.amazonaws.com/plot3d_utilities/iso65_64blocks.xyz

## Install Plot3D

In [ ]:
!pip install plot3d

In [ ]:
import os 
from plot3d import write_plot3D, read_plot3D, split_blocks, combine_nxnxn_cubes_mixed_pairs
from plot3d import connectivity_fast, plot_blocks, find_matching_faces
import pickle
import numpy as np 

cmc_p3d_file = 'iso65_64blocks.xyz'
cmc_p3d_bin = cmc_p3d_file.replace('.xyz','.bxyz')

## Find the connectivity between the blocks
This is required for CFD solving and for merging blocks 

In [ ]:
blocks = read_plot3D(cmc_p3d_file, binary = False)
# Block 1 is the blade O-Mesh k=0
# outer_faces, _ = get_outer_faces(blocks[0]) # lets check
face_matches, outer_faces_formatted = connectivity_fast(blocks)
test = np.array([(c['block1']['block_index'],c['block2']['block_index'])  for c in face_matches])
print(f'minimum block index: {test.min()}')
with open('connectivity.pickle','wb') as f:
    [m.pop('match',None) for m in face_matches] # Remove the dataframe
    pickle.dump({"face_matches":face_matches, "outer_faces":outer_faces_formatted},f)
write_plot3D(cmc_p3d_bin, blocks,binary = True)

In [ ]:
# Plot the original blocks 
plot_blocks(blocks) # this function accepts an array 

## Combine blocks
The code below automatically finds connected blocks and merges blocks together so 2x2x2 creates a rubix cube size 8 and 3x3x3 creates a rubix cube with 27 blocks.

In [ ]:

merged_face_matches = face_matches
merged_outer_faces = outer_faces_formatted
merged_block_only = blocks

for i in range(1): # Loop and find new pairs
    merged = combine_nxnxn_cubes_mixed_pairs(merged_block_only, merged_face_matches,cube_size=2)
    merged_blocks_only = [m[0] for m in merged]
    face_matches_2x2x2, outer_faces_formatted_2x2x2 = connectivity_fast(merged_blocks_only)
    write_plot3D("merged_2x2x2.xyz",merged_blocks_only,binary=False) 
    
    test = np.array([(c['block1']['block_index'],c['block2']['block_index'])  for c in merged_face_matches])
    print(f'minimum block index: {test.min()}')
    [m.pop('match',None) for m in merged_face_matches] # Remove the dataframe
    print(f'Merge pass {i} number of blocks {len(merged_blocks_only)}')

    with open('merged_connectivity.pickle','wb') as f:
        pickle.dump(
            {
                "face_matches":merged_face_matches,
                "outer_faces":merged_outer_faces,
                "blocks":merged_blocks_only
            },f)

In [ ]:
plot_blocks(merged_blocks_only)